Importing Libraries

In [25]:
import numpy as np
import sys
from time import time

Node Implementation:\
Node contains following five informations:
1. Parent node
2. State
3. Path Cost
4. Heuristic cost
5. Total cost = Path Cost + Heuristic cost 

It uses the following built-in functions:
1. __ hash__ : This provides the hash value for every node, which is required for the hashset
2. __ eq__ : To check if 2 nodes are equal 
3. __ neq__ : To check if 2 nodes are not equal 
4. __ str__ : To get string representation of state in node

In [26]:
class Node:
    def __init__(self, parent, state, pcost, hcost):
        
        self.parent = parent
        self.state = state
        # parent cost
        self.pcost = pcost
        # Heuristic cost
        self.hcost = hcost
        # Total Cost = Parent cost + Heuristic cost
        self.cost = pcost + hcost
    
    # Provides hsah value
    def __hash__(self):
        
        return hash(''.join(self.state.flatten()))
    
    # Get string representation
    def __str__(self):
        return str(self.state)
    
    # Check 2 nodes are equal
    def __eq__(self, other):
        
        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten())) 
    
    # Check 2 nodes are unequal
    def __neq__(self, other):
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten()))

PriorityQueue Implementation:\
It pops the node with having the lowest cost.\
It uses the following functions:
1. push = Add node to the queue
2. pop = Pop out node with the lowest cost
3. is_empty = Check whether queue is empty or not
4. __ len __ = gets the lengthof the queue
5. __ str __ = to get string representation of the queue

In [27]:
class PriorityQueue():
    
    def __init__(self):
        self.queue = []
        
    def push(self, node):
        self.queue.append(node)
    
    def pop(self):
        
        next_state = None
        state_cost = 10**18
        index = -1
        
        for i in range(len(self.queue)):
            # checking for the node with the mininmum cost
            if self.queue[i].cost<state_cost:
                state_cost = self.queue[i].cost
                index = i
        
        return self.queue.pop(index)
    
    def is_empty(self):
        
        return len(self.queue)==0
    
    def __str__(self):
        l = []
        for i in self.queue:
            l.append(i.state)
        
        return str(l)
    
    def __len__(self):
        return len(self.queue)

Environment Implementation:\
It has the following information:
1. Start state
2. Goal State
3. Actions that are to applied to node
4. Maximum depth of the solution

It makes use of the following functions:
1. generate_start_state : Genenates start state
2. get_start_state : Returns the start state
3. get_next_states : It returns all possible next states for a given current state 
4. reached_goal : Returns goal_state



In [28]:

class Environment():
    
    def __init__(self, depth = None, goal_state = None):
        #1 - Up, 2 - Down, 3 - Right, 4 - Left
        self.actions = [1,2,3,4] 
        self.goal_state = goal_state
        self.depth = depth
        self.start_state = self.generate_start_state()
    
    # generates new states
    def generate_start_state(self):
        
        past_state = self.goal_state
        i=0

        while i!= self.depth:
            new_states = self.get_next_states(past_state)
            choice = np.random.randint(low=0, high=len(new_states))
            
            if np.array_equal(new_states[choice], past_state):
                continue
            
            past_state = new_states[choice]
            i+=1
            
        return past_state
    
    # returns start state
    def get_start_state(self):
        return self.start_state
    
    # returns goal state
    def get_goal_state(self):
        return self.goal_state
    
    # generating new states from given state
    def get_next_states(self, state):
        
        space = (0,0)
        for i in range(3):
            for j in range(3):
                if state[i,j] == '_':
                    space = (i,j)
                    break
        
        new_states = []

        # Move Up if "_" is in row1 or row2 
        if space[0] > 0:
            new_state = np.copy(state)
            
            # Swapping of values
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]]  = new_state[space[0]-1, space[1]]
            new_state[space[0]-1, space[1]] = val
            
            new_states.append(new_state)

        # Move down if "_" is in row0 or row1
        if space[0] < 2: 
            new_state = np.copy(state)
            
            # Swapping of values
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]]  = new_state[space[0]+1, space[1]]
            new_state[space[0]+1, space[1]] = val
            
            new_states.append(new_state)
        
        # Move right if "_" is in column0 or column1
        if space[1]<2: 
            new_state = np.copy(state)
            
            # Swapping of values
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1]+1]
            new_state[space[0], space[1]+1] = val
            
            new_states.append(new_state)

        # Move left if "_" is in column1 or column2    
        if space[1] > 0: 
            new_state = np.copy(state)
            
            # Swapping of values
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1]-1]
            new_state[space[0], space[1]-1] = val
            
            new_states.append(new_state)
        
        return new_states
    
    # Checks whether it is goal state or not
    def reached_goal(self, state):
        
        for i in range(3):
            for j in range(3):
                if state[i,j] != self.goal_state[i,j]:
                    return False
        
        return True

Agent Implementation:\
It has the following information:
1. Frontier = priority queue which stores the nodes to be explored by the agent
2. explored = It stores the explored nodes
3. start_state = Stores the start state
4. goal_state = Stores the goal state
5. env = Stores the environment
6. goal_node = If found,then Stores the goal node
7. heuristic = stores the heuristic function

It has the following functions:
1. run = Function that explores the environment and finds the goal node
2. soln_depth = return depth at which solution is found
3. print_nodes = Print the path from start state to goal state
4. get_memory = return memory used to solve the question

In [29]:
class Agent:
    
    def __init__(self, env, heuristic):
        self.frontier = PriorityQueue()
        self.explored = dict()
        self.start_state = env.get_start_state()
        self.goal_state = env.get_goal_state()
        self.env = env
        self.goal_node = None
        self.heuristic = heuristic
    
    def run(self):
        init_node = Node(parent = None, state = self.start_state, pcost = 0, hcost=0)
        # pushing init_node to frontier
        self.frontier.push(init_node)
        steps = 0

        # Checks until the frontier queue is not empty
        while not self.frontier.is_empty():

            # Pops the node with the lowest code in queue ans assign it to curr-node 
            curr_node = self.frontier.pop()
            
            # Gets the next state for current node
            next_states = self.env.get_next_states(curr_node.state)

            #  if current node is already explored, skip it
            if hash(curr_node) in self.explored:
                continue

            # Adding curr_node to explored set
            self.explored[hash(curr_node)] = curr_node

            # If goal state is reached 
            if self.env.reached_goal(curr_node.state):
                
                self.goal_node = curr_node
                break
            goal_state = self.env.get_goal_state()

            l = []
            for state in next_states:

                hcost = self.heuristic(state, goal_state)
                node = Node(parent=curr_node, state=state, pcost=curr_node.pcost+1, hcost=hcost)
                self.frontier.push(node)
            steps += 1
        
        
        return steps, self.soln_depth()
    
    # Returns the number of nodes travelled
    def soln_depth(self):
        node = self.goal_node
        count = 0
        while node is not None:
            node = node.parent
            count+=1
        
        return count
    
    # Print nodes from starting node to goal node
    def print_nodes(self):
        
        node = self.goal_node
        l = []
        while node is not None:
            l.append(node)
            node = node.parent

        step = 1
        for node in l[::-1]:
            print("Step: ",step)
            print(node)
            step+=1
    
    # Returns memory used
    def get_memory(self):
        
        mem = len(self.frontier)*56 + len(self.explored)*56
        return mem

Heuristic 0\
This is a null heuristic, returns 0 for any state. \
Represents uniform cost search.

In [30]:
def heuristic0(curr_state, goal_state):
    return 0

Heuristic 1\
This heuristic counts the number of misplaced tiles

In [31]:
def heuristic1(curr_state, goal_state):
    
    count = 0
    for i in range(3):
        for j in range(3):
            if curr_state[i, j]!=goal_state[i,j]:
                count+=1
    
    return count

Heuristic 2\
This is the manhattan distance.\
 It returns the sum of the horizontal and vertical distances of the all marble in current state from center.

In [32]:
def heuristic2(curr_state, goal_state):
    
    dist = 0

    for i in range(3):
        for j in range(3):
            ele = curr_state[i, j]
            goal_i, goal_j = np.where(goal_state==ele)
            d = abs(goal_i[0] - i) + abs(goal_j[0] - j)
            dist += d
    
    return dist

In [33]:
depth = 500
goal_state = np.array([[1,2,3], [8,'_',4], [7,6,5]])
env = Environment(depth, goal_state)
print("Start State: ")
print(env.get_start_state())
print("Goal State: ")
print(goal_state)

Start State: 
[['_' '3' '6']
 ['1' '8' '5']
 ['4' '2' '7']]
Goal State: 
[['1' '2' '3']
 ['8' '_' '4']
 ['7' '6' '5']]


In [34]:
agent = Agent(env = env, heuristic = heuristic2)

In [35]:
agent.run()

(762, 23)

In [36]:
depths = np.arange(0,501,50)
goal_state = np.array([[1,2,3], [8,'_',4], [7,6,5]])
times_taken = {}
mems = {}
for depth in depths:
    
    time_taken = 0
    mem = 0
    for i in range(50):
        env = Environment(depth=depth, goal_state=goal_state)
        agent = Agent(env = env, heuristic = heuristic2)
        start_time = time()
        agent.run()
        end_time = time()
        time_taken+=end_time - start_time
        mem+=agent.get_memory()
    
    time_taken/=50
    mem = mem/50
    times_taken[depth] = time_taken
    mems[depth] = mem
    print(depth, time_taken, mem)

0 4.4002532958984376e-05 56.0
50 0.08828277587890625 25945.92
100 0.3768974447250366 70851.2
150 0.27368505477905275 73069.92
200 0.5049377107620239 95510.24
250 0.49290042877197265 107280.32
300 0.7130078077316284 124741.12
350 0.5293329429626464 135605.12
400 0.9476835918426514 144844.0
450 0.5781193876266479 116832.8
500 0.5433288621902466 124088.16
